# Installing necessary libraries

In [12]:
!pip install sparsezoo

In [13]:
!pip install deepsparse

  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [22 lines of output]
  Loaded version 1.8.0 from C:\Users\ayush\AppData\Local\Temp\pip-install-jjk1pjky\deepsparse_a2fd6f60aa0749e1971e4b9cf63c78ba\src\deepsparse\version.py
  Traceback (most recent call last):
    File "C:\Users\ayush\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\ayush\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\ayush\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
      return hook(config_settings)
             ^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\ayush\AppData\Local\Temp\pip-b


  Using cached deepsparse-1.8.0.tar.gz (46.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


In [14]:
!pip install onnx


In [15]:
!pip install sparsezoo onnx


# Extracting Weight Matrices

In [58]:
from sparsezoo import Model
import os
import onnx
import numpy as np
from onnx import numpy_helper

# Load the model using the specified stub
stub = "zoo:resnet_v1-50-imagenet-pruned95_quantized"
model = Model(stub)

# Access the download path of the model
download_path = model.path  # This is where the model will be downloaded
onnx_model_file = model.onnx_model.path

# Download the model files if they don't already exist
if os.path.isdir(download_path) and os.listdir(download_path):
    print(f"Model already downloaded at: {onnx_model_file}")
else:
    # Download the model files
    model.download()

# Print model information
print(f"Model downloaded at: {onnx_model_file}")

# Load the ONNX model
onnx_model = onnx.load(onnx_model_file)

# Function to calculate sparsity and features
def calculate_features(weight_matrix):
    print(f"Calculating features for matrix with shape: {weight_matrix.shape}")
    
    if weight_matrix.ndim == 0:  # Scalar
        print(f"Skipping feature calculation for scalar tensor with shape {weight_matrix.shape}")
        return {
            'density': 1.0 if weight_matrix != 0 else 0.0,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }
    elif weight_matrix.ndim == 1:  # 1D array
        nnz = np.count_nonzero(weight_matrix)
        density = nnz / weight_matrix.size
        print(f"Skipping detailed feature calculation for 1D tensor with shape {weight_matrix.shape}")
        return {
            'density': density,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    N = weight_matrix.shape[0]  # Number of rows
    nnz = np.count_nonzero(weight_matrix)
    
    # Calculate density based on the number of elements in the matrix
    if weight_matrix.ndim == 2:
        density = nnz / (N ** 2)  # Density as NNZ/N^2 for 2D matrices
    else:
        density = nnz / weight_matrix.size  # General density for other dimensions

    if weight_matrix.ndim == 2:
        nnz_per_row = np.count_nonzero(weight_matrix, axis=1)
        bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in weight_matrix]
    elif weight_matrix.ndim == 4:
        reshaped_matrix = weight_matrix.reshape(weight_matrix.shape[0], -1)
        nnz_per_row = np.count_nonzero(reshaped_matrix, axis=1)
        bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in reshaped_matrix]
    else:
        print(f"Skipping detailed feature calculation for tensor with shape {weight_matrix.shape}")
        return {
            'density': density,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    nnz_min = np.min(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_max = np.max(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_avg = np.mean(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_sd = np.std(nnz_per_row) if nnz_per_row.size > 0 else 0

    bw_min = np.min(bw) if bw else 0
    bw_max = np.max(bw) if bw else 0
    bw_avg = np.mean(bw) if bw else 0
    bw_sd = np.std(bw) if bw else 0

    scatter = nnz_per_row / (np.array(bw) + 1e-9)
    scatter_avg = np.mean(scatter) if scatter.size > 0 else 0
    scatter_sd = np.std(scatter) if scatter.size > 0 else 0

    clustering = []
    for row in reshaped_matrix if weight_matrix.ndim == 4 else weight_matrix:
        groups = 0
        prev_nonzero = -2
        for idx in np.nonzero(row)[0]:
            if idx != prev_nonzero + 1:
                groups += 1
            prev_nonzero = idx
        clustering.append(groups / (np.count_nonzero(row) + 1e-9))
    clustering_avg = np.mean(clustering) if clustering else 0

    return {
        'density': density,
        'nnz_min': nnz_min,
        'nnz_max': nnz_max,
        'nnz_avg': nnz_avg,
        'nnz_sd': nnz_sd,
        'bw_min': bw_min,
        'bw_max': bw_max,
        'bw_avg': bw_avg,
        'bw_sd': bw_sd,
        'scatter_avg': scatter_avg,
        'scatter_sd': scatter_sd,
        'clustering_avg': clustering_avg
    }


# Print the layers, their weights, and calculated features
for tensor in onnx_model.graph.initializer:
    print(f"Layer: {tensor.name}")
    print(f"Shape: {tensor.dims}")
    weights = numpy_helper.to_array(tensor)
    print(f"Weights: {weights}\n")

    # Calculate features
    features = calculate_features(weights)
    
    # Check if any of the calculated features are None and skip the layer if so
    if any(value is None for value in features.values()):
        print(f"Skipping layer {tensor.name} due to None values in features.\n")
        continue  # Skip this layer if any feature is None
    
    print(f"Features for {tensor.name}:")
    for key, value in features.items():
        print(f"  {key}: {value}")
    print("\n")

# If you want to extract and handle weights and their features more specifically,
# you can store them in a dictionary.
weights_dict = {tensor.name: numpy_helper.to_array(tensor) for tensor in onnx_model.graph.initializer}

# Example: Print specific layer weights and features
for layer_name, weights in weights_dict.items():
    print(f"Layer: {layer_name}, Weights: {weights.shape}")
    print(weights)

    # Calculate features
    features = calculate_features(weights)
    
    # Check if any of the calculated features are None and skip the layer if so
    if any(value is None for value in features.values()):
        print(f"Skipping layer {layer_name} due to None values in features.\n")
        continue  # Skip this layer if any feature is None
    
    print(f"Features for {layer_name}:")
    for key, value in features.items():
        print(f"  {key}: {value}")
    print("\n")


Model already downloaded at: C:\Users\ayush\.cache\sparsezoo\neuralmagic\resnet_v1-50-imagenet-pruned95_quantized\model.onnx
Model downloaded at: C:\Users\ayush\.cache\sparsezoo\neuralmagic\resnet_v1-50-imagenet-pruned95_quantized\model.onnx
Layer: classifier.fc.weight
Shape: [1000, 2048]
Weights: [[-0.07376207  0.         -0.         ...  0.          0.
   0.        ]
 [ 0.12108373  0.         -0.         ...  0.          0.
   0.        ]
 [-0.          0.         -0.1097383  ...  0.          0.
   0.        ]
 ...
 [-0.0622465   0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.         -0.04466717 ...  0.          0.
   0.        ]
 [ 0.11173791  0.          0.05249852 ...  0.          0.
   0.        ]]

Calculating features for matrix with shape: (1000, 2048)
Features for classifier.fc.weight:
  density: 0.256699
  nnz_min: 102
  nnz_max: 426
  nnz_avg: 256.699
  nnz_sd: 45.01991113940586
  bw_min: 1962
  bw_max: 2045
  bw_avg: 2030.729
  bw_sd: 10.750700

Features for Conv_315.weight_quantized:
  density: 1.0
  nnz_min: 128
  nnz_max: 128
  nnz_avg: 128.0
  nnz_sd: 0.0
  bw_min: 128
  bw_max: 128
  bw_avg: 128.0
  bw_sd: 0.0
  scatter_avg: 0.9999999999921876
  scatter_sd: 0.0
  clustering_avg: 0.0078124999999389655


Layer: Conv_315.bias_quantized
Shape: [512]
Weights: [    0  1145   107  4142   556  -685  3516     0  2113  1576   316  1007
    87  1562   849  1408   584 -2991  5610   337   387   890     0  -480
     0  -171   710   589     0     0     0    83  1646     0     0   304
    32     0   987  1087   -10     0  1093     0   677  -755     0     0
 -2163     0     0   354   861 -3511   367  -504     5  -290  2831  1604
   150     0   524   159  -749 -1299  2007  2839   895     0  -438     0
 -4035  -290   240  -814 -1171   362   328  2565     0     0   375     0
     0  -421   657 -5102     0   694   847     0     0   615  1320     0
  2303   426   808     0  3375  1154  1818  -572     0   735     0   176
 -2509   514  -230 -552

Features for Conv_454.weight_quantized:
  density: 1.0
  nnz_min: 1152
  nnz_max: 1152
  nnz_avg: 1152.0
  nnz_sd: 0.0
  bw_min: 1152
  bw_max: 1152
  bw_avg: 1152.0
  bw_sd: 0.0
  scatter_avg: 0.9999999999991317
  scatter_sd: 2.220446049250313e-16
  clustering_avg: 0.0008680555555548018


Layer: Conv_454.bias_quantized
Shape: [128]
Weights: [  4329    240     40  -1067   7522   4806   2126  11119   1103  -5146
   2364   7606  -2269 -10326  11227    795      0  10608  -5158   -464
   2992   6623   6374    386  -8555   5345  -3145   2717  -1988   1203
  -2411      1  13540    174  -4640   -386  -4076   9148  -3940   5539
  -3127   1008    247   1872  -5772   4074   2774    265     63  15920
  -2438 -11887  -3734  -6886   1226  -4164   2941   9156   9512   1832
   5622  -2764  -3919   4658  15357   2913   4140   -834   1207   3027
  -2358  -5092   2292   4785 -13144     18   2053  -2336   2199   7003
  -4353   4140   1044  -2603   5401  -2578   2477  10833   3768   8696
   7727   3627   

Features for Conv_594.weight_quantized:
  density: 1.0
  nnz_min: 2304
  nnz_max: 2304
  nnz_avg: 2304.0
  nnz_sd: 0.0
  bw_min: 2304
  bw_max: 2304
  bw_avg: 2304.0
  bw_sd: 0.0
  scatter_avg: 0.9999999999995662
  scatter_sd: 2.220446049250313e-16
  clustering_avg: 0.00043402777777758954


Layer: Conv_594.bias_quantized
Shape: [256]
Weights: [-2008  -970  3422   231  2858 -1612  3452  2784  -825  1680  -669  3679
  2359   474  1274 -1810  2731  3964  1502  2671  3207 -1177  3297  3228
 -1697  3580   697   572  2767  1333  3249  3888  1753    13  -258   726
 -2671  2014   422  1360 -3292  -223 -1073 -1144 -1663   197    75  3175
  -228  -100 -3674  2804  2792  1731 -1057 -2386  -368  2801  1383  -506
  -713  2884  2118  2783   624  3136  -147  2692   958  1074 -1509  2279
 -1545  2762  1436   474   673   568   204   520  -153    75   308  1158
  3491  3449 -3362 -1436 -3259    28   262   258  2262 -1019  1337  2794
  1149   682  1115  5733  3891   409  1226 -1546  -151  2046 -1529  335

Features for Conv_686.weight_quantized:
  density: 1.0
  nnz_min: 2304
  nnz_max: 2304
  nnz_avg: 2304.0
  nnz_sd: 0.0
  bw_min: 2304
  bw_max: 2304
  bw_avg: 2304.0
  bw_sd: 0.0
  scatter_avg: 0.9999999999995662
  scatter_sd: 2.220446049250313e-16
  clustering_avg: 0.00043402777777758954


Layer: Conv_686.bias_quantized
Shape: [256]
Weights: [  -312      0  -1641   1840  -1080  -2160  -1151   -855  -3442   -257
      0   5130  -1911   -664   1054   4368  -3003   2561    -25  -3931
    632   -312  -3642  -6078   1149   -163      0  -1471  -3486    -59
   -922      0      0  -3361      0   3131    487      0  -1341  -2006
    705   -245  -3528  -2134   -540  -1107  -1809  -7127      0  -6552
  -1511  -1146      0      0  -4429   2837  -1579      0   1702  -4743
  -3194   4390  -2617      0   8221    822  -3525  -3273      0  -4404
  -2445  -2777   3600  -1674    282      0      0  -2159    -28      0
      0     42  -1752  -5911   -650    109   -989  -1268 -10549      0
      0  -1130  

...

Features for Conv_896.weight_quantized:
  density: 1.0
  nnz_min: 2304
  nnz_max: 2304
  nnz_avg: 2304.0
  nnz_sd: 0.0
  bw_min: 2304
  bw_max: 2304
  bw_avg: 2304.0
  bw_sd: 0.0
  scatter_avg: 0.9999999999995662
  scatter_sd: 2.220446049250313e-16
  clustering_avg: 0.00043402777777758954


Layer: Conv_896.bias_quantized, Weights: (256,)
[  -78   184   814     0 -6360   996     0     0     0     0     0     0
   530     0     0   469  -356 -1273     0  -840     0     0  -997 -1103
   709     0     0 -1347 -1501  2702     0  -132  -811 -4289     0 -1634
  1299  -781 -3255     0  2663   853 -3153  -141 -1338  1245  -285     0
     0 -7787 -2073     0  2488 -3076   -48   627  -491 -3181     0 -1385
 -1009 -5773     0   103     0 -5758     0  2236  2710     0 -1785     0
   596 -1859  4358 -1064  3659   666 -1249     0   184  -338     0  -422
     0 -2243   -95  2090 -2096  2531     0   369  1151 -2237  1498   375
   328 -2156  1483     0     0   998 -2054     0     0 -1755     0  -735
  4

Features for Conv_1013.weight_quantized:
  density: 1.0
  nnz_min: 1024
  nnz_max: 1024
  nnz_avg: 1024.0
  nnz_sd: 0.0
  bw_min: 1024
  bw_max: 1024
  bw_avg: 1024.0
  bw_sd: 0.0
  scatter_avg: 0.9999999999990231
  scatter_sd: 3.3306690738754696e-16
  clustering_avg: 0.000976562499999046


Layer: Conv_1013.bias_quantized, Weights: (512,)
[  876 -1204   788 -4608  -438   208  -430  1448 -3987 -1064 -1946     0
     0 -1555   508   122   -79   547 -1191 -1824   360  -738  1157  -350
 -2957  -673  -315 -1861 -1074  -769     0   -60  -195 -1872  -508 -1321
 -1868  -899  -385 -2042 -2275   706  -195   134     0  -577 -1439   601
 -5189  -434 -1193 -1991  -494 -2779 -1773   375 -2548    10  -584  -901
   137 -2997  -785  -699   711  -708 -1501  1250   816 -1203     0   481
   503  -469 -1709 -2496   867 -1565 -2951 -1940 -1086  -115  -343 -1522
  -951  -562   620  -312 -1717 -1275 -1688   301 -1960  -420   194   -38
 -1291  1132 -2778   739 -1473   680  1512  -614 -1410  -308   616   686
  

Features for Conv_1059.weight_quantized:
  density: 1.0
  nnz_min: 512
  nnz_max: 512
  nnz_avg: 512.0
  nnz_sd: 0.0
  bw_min: 512
  bw_max: 512
  bw_avg: 512.0
  bw_sd: 0.0
  scatter_avg: 0.9999999999980465
  scatter_sd: 4.440892098500626e-16
  clustering_avg: 0.0019531249999961845


Layer: Conv_1059.bias_quantized, Weights: (2048,)
[ 581    0 -672 ...    0    0    0]
Calculating features for matrix with shape: (2048,)
Skipping detailed feature calculation for 1D tensor with shape (2048,)
Skipping layer Conv_1059.bias_quantized due to None values in features.

Layer: Conv_1081.weight_quantized, Weights: (2048, 1024, 1, 1)
[[[[128]]

  [[128]]

  [[128]]

  ...

  [[128]]

  [[121]]

  [[128]]]


 [[[128]]

  [[128]]

  [[128]]

  ...

  [[128]]

  [[128]]

  [[128]]]


 [[[128]]

  [[128]]

  [[128]]

  ...

  [[128]]

  [[128]]

  [[128]]]


 ...


 [[[128]]

  [[128]]

  [[128]]

  ...

  [[128]]

  [[128]]

  [[128]]]


 [[[128]]

  [[128]]

  [[128]]

  ...

  [[128]]

  [[128]]



Features for Conv_1151.weight_quantized:
  density: 1.0
  nnz_min: 512
  nnz_max: 512
  nnz_avg: 512.0
  nnz_sd: 0.0
  bw_min: 512
  bw_max: 512
  bw_avg: 512.0
  bw_sd: 0.0
  scatter_avg: 0.9999999999980465
  scatter_sd: 4.440892098500626e-16
  clustering_avg: 0.0019531249999961845


Layer: Conv_1151.bias_quantized, Weights: (2048,)
[  209     0 -1182 ...     0     0     0]
Calculating features for matrix with shape: (2048,)
Skipping detailed feature calculation for 1D tensor with shape (2048,)
Skipping layer Conv_1151.bias_quantized due to None values in features.

Layer: Conv_1175.weight_quantized, Weights: (512, 2048, 1, 1)
[[[[128]]

  [[128]]

  [[128]]

  ...

  [[128]]

  [[128]]

  [[128]]]


 [[[128]]

  [[128]]

  [[128]]

  ...

  [[128]]

  [[128]]

  [[128]]]


 [[[128]]

  [[128]]

  [[128]]

  ...

  [[128]]

  [[128]]

  [[128]]]


 ...


 [[[128]]

  [[128]]

  [[128]]

  ...

  [[128]]

  [[128]]

  [[128]]]


 [[[128]]

  [[128]]

  [[120]]

  ...

  [[128]]

  [[12

Features for Conv_1221.weight_quantized:
  density: 1.0
  nnz_min: 512
  nnz_max: 512
  nnz_avg: 512.0
  nnz_sd: 0.0
  bw_min: 512
  bw_max: 512
  bw_avg: 512.0
  bw_sd: 0.0
  scatter_avg: 0.9999999999980465
  scatter_sd: 4.440892098500626e-16
  clustering_avg: 0.0019531249999961845


Layer: Conv_1221.bias_quantized, Weights: (2048,)
[-864    0 -231 ...    0    0    0]
Calculating features for matrix with shape: (2048,)
Skipping detailed feature calculation for 1D tensor with shape (2048,)
Skipping layer Conv_1221.bias_quantized due to None values in features.



# First Run (YOLOV5 Models)

In [30]:
import os
import onnx
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from onnx import numpy_helper
from sparsezoo import Model

# Function to calculate sparsity and features
# Function to calculate sparsity and features
def calculate_features(weight_matrix):
    if weight_matrix.size == 0:  # Check if the matrix size is zero
        return {
            'density': None,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    if weight_matrix.ndim == 0:  # Scalar
        return {
            'density': 1.0 if weight_matrix != 0 else 0.0,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }
    elif weight_matrix.ndim == 1:  # 1D array
        nnz = np.count_nonzero(weight_matrix)
        density = nnz / weight_matrix.size if weight_matrix.size > 0 else 0
        return {
            'density': density,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    N = weight_matrix.shape[0]
    nnz = np.count_nonzero(weight_matrix)
    
    # Calculate density
    if weight_matrix.ndim == 2:
        density = nnz / (N ** 2) if N > 0 else 0
    else:
        density = nnz / weight_matrix.size

    if weight_matrix.ndim == 2:
        nnz_per_row = np.count_nonzero(weight_matrix, axis=1)
        bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in weight_matrix]
    elif weight_matrix.ndim == 4:
        reshaped_matrix = weight_matrix.reshape(weight_matrix.shape[0], -1)
        nnz_per_row = np.count_nonzero(reshaped_matrix, axis=1)
        bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in reshaped_matrix]
    else:
        return {
            'density': density,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    nnz_min = np.min(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_max = np.max(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_avg = np.mean(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_sd = np.std(nnz_per_row) if nnz_per_row.size > 0 else 0

    bw_min = np.min(bw) if bw else 0
    bw_max = np.max(bw) if bw else 0
    bw_avg = np.mean(bw) if bw else 0
    bw_sd = np.std(bw) if bw else 0

    scatter = nnz_per_row / (np.array(bw) + 1e-9)
    scatter_avg = np.mean(scatter) if scatter.size > 0 else 0
    scatter_sd = np.std(scatter) if scatter.size > 0 else 0

    clustering = []
    for row in reshaped_matrix if weight_matrix.ndim == 4 else weight_matrix:
        groups = 0
        prev_nonzero = -2
        for idx in np.nonzero(row)[0]:
            if idx != prev_nonzero + 1:
                groups += 1
            prev_nonzero = idx
        clustering.append(groups / (np.count_nonzero(row) + 1e-9))
    clustering_avg = np.mean(clustering) if clustering else 0

    return {
        'density': density,
        'nnz_min': nnz_min,
        'nnz_max': nnz_max,
        'nnz_avg': nnz_avg,
        'nnz_sd': nnz_sd,
        'bw_min': bw_min,
        'bw_max': bw_max,
        'bw_avg': bw_avg,
        'bw_sd': bw_sd,
        'scatter_avg': scatter_avg,
        'scatter_sd': scatter_sd,
        'clustering_avg': clustering_avg
    }


def extract_metrics_from_models(model_stubs, sample_size=None):
    metrics_list = []

    for stub, accuracy in model_stubs:
        print(f"Processing model: {stub}")
        model = Model(stub)
        onnx_model_file = model.onnx_model.path
        onnx_model = onnx.load(onnx_model_file)

        for tensor in onnx_model.graph.initializer:
            weights = numpy_helper.to_array(tensor)
            features = calculate_features(weights)

            if any(value is None for value in features.values()):
                continue  # Skip this layer if any feature is None

            features['layer'] = tensor.name
            features['accuracy'] = accuracy
            features['model'] = stub
            metrics_list.append(features)
    
    df_metrics = pd.DataFrame(metrics_list)
    
    if sample_size and len(df_metrics) > sample_size:
        df_metrics = df_metrics.sample(n=sample_size, random_state=42)  # Sample data to reduce size
    
    return df_metrics


# Function to plot the distribution of metrics and save them as images
def plot_metrics_distribution(df_metrics, output_dir="plots"):
    metrics = ['density', 'nnz_min', 'nnz_max', 'nnz_avg', 'nnz_sd', 'bw_min', 'bw_max', 'bw_avg', 'bw_sd', 'scatter_avg', 'scatter_sd', 'clustering_avg']
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for metric in metrics:
        # Check if the column exists and has valid (non-None) data
        if metric in df_metrics.columns and df_metrics[metric].notna().any():
            plt.figure(figsize=(10, 6))
            sns.histplot(df_metrics[metric].dropna(), kde=True, bins=50)
            plt.title(f'Distribution of {metric}')
            plt.xlabel(metric)
            plt.ylabel('Frequency')
            plt.savefig(os.path.join(output_dir, f'{metric}_distribution.png'))
            plt.close()  # Close the plot to free memory

# Function to plot correlation of metrics with accuracy and save them as images
def plot_correlation_with_accuracy(df_metrics, output_dir="plots"):
    metrics = ['density', 'nnz_min', 'nnz_max', 'nnz_avg', 'nnz_sd', 'bw_min', 'bw_max', 'bw_avg', 'bw_sd', 'scatter_avg', 'scatter_sd', 'clustering_avg']
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for metric in metrics:
        # Check if the column exists and has valid (non-None) data
        if metric in df_metrics.columns and df_metrics[metric].notna().any():
            plt.figure(figsize=(10, 6))
            sns.regplot(x=df_metrics[metric], y=df_metrics['accuracy'], ci=None, scatter_kws={'s': 50})
            plt.title(f'Correlation of {metric} with Accuracy')
            plt.xlabel(metric)
            plt.ylabel('Accuracy')
            plt.savefig(os.path.join(output_dir, f'{metric}_correlation.png'))
            plt.close()  # Close the plot to free memory

# Example model stubs with their corresponding accuracies
model_stubs = [
    ("zoo:yolov5-n-coco-pruned40_quantized", 45.1),
    ("zoo:yolov5-n-voc_coco-pruned30.4block_quantized", 71.3),
    ("zoo:yolov5-n6-voc_coco-pruned55_quantized", 80.6),
    ("zoo:yolov5-n6-coco-pruned55_quantized", 52.8),
    ("zoo:yolov5-s-coco-pruned85_quantized", 53.4),
    ("zoo:yolov5-s-voc_coco-pruned_quantized", 36.7),
    ("zoo:yolov5-s6-voc_coco-pruned65_quantized", 86.8),
    ("zoo:yolov5-s6-coco-pruned65_quantized", 61.5),
    ("zoo:yolov5-m-coco-pruned70_quantized", 63.2),
    ("zoo:yolov5-m-voc_coco-pruned70_quantized", 89.5)
    # Add more models and their accuracies here
]

# Extract metrics from the models
df_metrics = extract_metrics_from_models(model_stubs, sample_size=1000)  # Add sample_size to limit data if necessary

# Plot the distribution of metrics and save them as images
plot_metrics_distribution(df_metrics, output_dir="plots")

# Plot the correlation of metrics with accuracy and save them as images
plot_correlation_with_accuracy(df_metrics, output_dir="plots")



Processing model: zoo:yolov5-n-coco-pruned40_quantized
Processing model: zoo:yolov5-n-voc_coco-pruned30.4block_quantized
Processing model: zoo:yolov5-n6-voc_coco-pruned55_quantized
Processing model: zoo:yolov5-n6-coco-pruned55_quantized
Processing model: zoo:yolov5-s-coco-pruned85_quantized
Processing model: zoo:yolov5-s-voc_coco-pruned_quantized
Processing model: zoo:yolov5-s6-voc_coco-pruned65_quantized
Processing model: zoo:yolov5-s6-coco-pruned65_quantized
Processing model: zoo:yolov5-m-coco-pruned70_quantized
Processing model: zoo:yolov5-m-voc_coco-pruned70_quantized


In [31]:
import os

# List all files in the 'plots' directory
output_dir = "plots"
files = os.listdir(output_dir)
files


['bw_avg_correlation.png',
 'bw_avg_distribution.png',
 'bw_max_correlation.png',
 'bw_max_distribution.png',
 'bw_min_correlation.png',
 'bw_min_distribution.png',
 'bw_sd_correlation.png',
 'bw_sd_distribution.png',
 'clustering_avg_correlation.png',
 'clustering_avg_distribution.png',
 'density_correlation.png',
 'density_distribution.png',
 'nnz_avg_correlation.png',
 'nnz_avg_distribution.png',
 'nnz_max_correlation.png',
 'nnz_max_distribution.png',
 'nnz_min_correlation.png',
 'nnz_min_distribution.png',
 'nnz_sd_correlation.png',
 'nnz_sd_distribution.png',
 'scatter_avg_correlation.png',
 'scatter_avg_distribution.png',
 'scatter_sd_correlation.png',
 'scatter_sd_distribution.png']

In [32]:
import shutil

# Create a zip file containing all images
shutil.make_archive('plots_archive', 'zip', 'plots')


'C:\\Users\\ayush\\Desktop\\Msc\\plots_archive.zip'

In [33]:
from IPython.display import FileLink

# Create a download link for the zip file
FileLink('plots_archive.zip')


C:\Users\ayush\Desktop\Msc\plots_archive.zip

In [ ]:
# Trying to plot the graphs in console
# # import os
# # import onnx
# # import numpy as np
# # import pandas as pd
# # import seaborn as sns
# # import matplotlib.pyplot as plt
# # from onnx import numpy_helper
# # from sparsezoo import Model

# # # Function to calculate sparsity and features
# # def calculate_features(weight_matrix):
# #     if weight_matrix.size == 0:  # Check if the matrix size is zero
# #         return {
# #             'density': None,
# #             'nnz_min': None,
# #             'nnz_max': None,
# #             'nnz_avg': None,
# #             'nnz_sd': None,
# #             'bw_min': None,
# #             'bw_max': None,
# #             'bw_avg': None,
# #             'bw_sd': None,
# #             'scatter_avg': None,
# #             'scatter_sd': None,
# #             'clustering_avg': None
# #         }

# #     if weight_matrix.ndim == 0:  # Scalar
# #         return {
# #             'density': 1.0 if weight_matrix != 0 else 0.0,
# #             'nnz_min': None,
# #             'nnz_max': None,
# #             'nnz_avg': None,
# #             'nnz_sd': None,
# #             'bw_min': None,
# #             'bw_max': None,
# #             'bw_avg': None,
# #             'bw_sd': None,
# #             'scatter_avg': None,
# #             'scatter_sd': None,
# #             'clustering_avg': None
# #         }
# #     elif weight_matrix.ndim == 1:  # 1D array
# #         nnz = np.count_nonzero(weight_matrix)
# #         density = nnz / weight_matrix.size if weight_matrix.size > 0 else 0
# #         return {
# #             'density': density,
# #             'nnz_min': None,
# #             'nnz_max': None,
# #             'nnz_avg': None,
# #             'nnz_sd': None,
# #             'bw_min': None,
# #             'bw_max': None,
# #             'bw_avg': None,
# #             'bw_sd': None,
# #             'scatter_avg': None,
# #             'scatter_sd': None,
# #             'clustering_avg': None
# #         }

# #     N = weight_matrix.shape[0]
# #     nnz = np.count_nonzero(weight_matrix)
    
# #     # Calculate density
# #     if weight_matrix.ndim == 2:
# #         density = nnz / (N ** 2) if N > 0 else 0
# #     else:
# #         density = nnz / weight_matrix.size

# #     if weight_matrix.ndim == 2:
# #         nnz_per_row = np.count_nonzero(weight_matrix, axis=1)
# #         bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in weight_matrix]
# #     elif weight_matrix.ndim == 4:
# #         reshaped_matrix = weight_matrix.reshape(weight_matrix.shape[0], -1)
# #         nnz_per_row = np.count_nonzero(reshaped_matrix, axis=1)
# #         bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in reshaped_matrix]
# #     else:
# #         return {
# #             'density': density,
# #             'nnz_min': None,
# #             'nnz_max': None,
# #             'nnz_avg': None,
# #             'nnz_sd': None,
# #             'bw_min': None,
# #             'bw_max': None,
# #             'bw_avg': None,
# #             'bw_sd': None,
# #             'scatter_avg': None,
# #             'scatter_sd': None,
# #             'clustering_avg': None
# #         }

# #     nnz_min = np.min(nnz_per_row) if nnz_per_row.size > 0 else 0
# #     nnz_max = np.max(nnz_per_row) if nnz_per_row.size > 0 else 0
# #     nnz_avg = np.mean(nnz_per_row) if nnz_per_row.size > 0 else 0
# #     nnz_sd = np.std(nnz_per_row) if nnz_per_row.size > 0 else 0

# #     bw_min = np.min(bw) if bw else 0
# #     bw_max = np.max(bw) if bw else 0
# #     bw_avg = np.mean(bw) if bw else 0
# #     bw_sd = np.std(bw) if bw else 0

# #     scatter = nnz_per_row / (np.array(bw) + 1e-9)
# #     scatter_avg = np.mean(scatter) if scatter.size > 0 else 0
# #     scatter_sd = np.std(scatter) if scatter.size > 0 else 0

# #     clustering = []
# #     for row in reshaped_matrix if weight_matrix.ndim == 4 else weight_matrix:
# #         groups = 0
# #         prev_nonzero = -2
# #         for idx in np.nonzero(row)[0]:
# #             if idx != prev_nonzero + 1:
# #                 groups += 1
# #             prev_nonzero = idx
# #         clustering.append(groups / (np.count_nonzero(row) + 1e-9))
# #     clustering_avg = np.mean(clustering) if clustering else 0

# #     return {
# #         'density': density,
# #         'nnz_min': nnz_min,
# #         'nnz_max': nnz_max,
# #         'nnz_avg': nnz_avg,
# #         'nnz_sd': nnz_sd,
# #         'bw_min': bw_min,
# #         'bw_max': bw_max,
# #         'bw_avg': bw_avg,
# #         'bw_sd': bw_sd,
# #         'scatter_avg': scatter_avg,
# #         'scatter_sd': scatter_sd,
# #         'clustering_avg': clustering_avg
# #     }

# # # Function to extract metrics from models
# # def extract_metrics_from_models(model_stubs, sample_size=None):
# #     metrics_list = []

# #     for stub, accuracy in model_stubs:
# #         print(f"Processing model: {stub}")
# #         model = Model(stub)
# #         onnx_model_file = model.onnx_model.path
# #         onnx_model = onnx.load(onnx_model_file)

# #         for tensor in onnx_model.graph.initializer:
# #             weights = numpy_helper.to_array(tensor)
# #             features = calculate_features(weights)

# #             if any(value is None for value in features.values()):
# #                 continue  # Skip this layer if any feature is None

# #             features['layer'] = tensor.name
# #             features['accuracy'] = accuracy
# #             features['model'] = stub
# #             metrics_list.append(features)
    
# #     df_metrics = pd.DataFrame(metrics_list)
    
# #     if sample_size and len(df_metrics) > sample_size:
# #         df_metrics = df_metrics.sample(n=sample_size, random_state=42)  # Sample data to reduce size
    
# #     return df_metrics

# # Function to plot the distribution of metrics
# def plot_metrics_distribution(df_metrics):
#     metrics = ['density', 'nnz_min', 'nnz_max', 'nnz_avg', 'nnz_sd', 'bw_min', 'bw_max', 'bw_avg', 'bw_sd', 'scatter_avg', 'scatter_sd', 'clustering_avg']
    
#     for metric in metrics:
#         plt.figure(figsize=(10, 6))
#         sns.histplot(df_metrics[metric].dropna(), kde=True, bins=50)
#         plt.title(f'Distribution of {metric}')
#         plt.xlabel(metric)
#         plt.ylabel('Frequency')
#         plt.show()

# # Function to plot correlation of metrics with accuracy
# def plot_correlation_with_accuracy(df_metrics):
#     metrics = ['density', 'nnz_min', 'nnz_max', 'nnz_avg', 'nnz_sd', 'bw_min', 'bw_max', 'bw_avg', 'bw_sd', 'scatter_avg', 'scatter_sd', 'clustering_avg']
    
#     for metric in metrics:
#         plt.figure(figsize=(10, 6))
#         sns.regplot(x=df_metrics[metric], y=df_metrics['accuracy'], ci=None, scatter_kws={'s': 50})
#         plt.title(f'Correlation of {metric} with Accuracy')
#         plt.xlabel(metric)
#         plt.ylabel('Accuracy')
#         plt.show()

# # # Example model stubs with their corresponding accuracies
# # model_stubs = [
# #     ("zoo:yolo_v3-spp-coco-pruned.4block_quantized", 60.5),
# #     ("zoo:vgg-16-imagenet-pruned", 70.8),
# #     ("zoo:inception_v3-imagenet-pruned", 76.6),
# #     ("zoo:yolov8-m-coco-pruned80_quantized", 65.2),
# #     ("zoo:yolov5-m6-coco-pruned75_quantized", 67.3),
# #     ("zoo:vgg-19-imagenet-pruned", 71.7),
# #     ("zoo:yolov5-s6-voc_coco-pruned65_quantized", 86.8),
# #     ("zoo:vgg-11-imagenet-pruned", 68.3),
# #     ("zoo:yolov5-x-voc_coco-pruned70.4block_quantized", 90.5),
# #     ("zoo:yolov8-n-coco-pruned49_quantized", 50.1)
# #     # Add more models and their accuracies here
# # ]

# # # Extract metrics from the models
# # df_metrics = extract_metrics_from_models(model_stubs, sample_size=1000)  # Add sample_size to limit data if necessary

# # # Plot the distribution of metrics
# # plot_metrics_distribution(df_metrics)

# # # Plot the correlation of metrics with accuracy
# # plot_correlation_with_accuracy(df_metrics)


In [ ]:
# import os
# import onnx
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# from onnx import numpy_helper
# from sparsezoo import Model

# # Function to calculate sparsity and features
# def calculate_features(weight_matrix):
#     if weight_matrix.size == 0:  # Check if the matrix size is zero
#         return {
#             'density': None,
#             'nnz_min': None,
#             'nnz_max': None,
#             'nnz_avg': None,
#             'nnz_sd': None,
#             'bw_min': None,
#             'bw_max': None,
#             'bw_avg': None,
#             'bw_sd': None,
#             'scatter_avg': None,
#             'scatter_sd': None,
#             'clustering_avg': None
#         }

#     if weight_matrix.ndim == 0:  # Scalar
#         return {
#             'density': 1.0 if weight_matrix != 0 else 0.0,
#             'nnz_min': None,
#             'nnz_max': None,
#             'nnz_avg': None,
#             'nnz_sd': None,
#             'bw_min': None,
#             'bw_max': None,
#             'bw_avg': None,
#             'bw_sd': None,
#             'scatter_avg': None,
#             'scatter_sd': None,
#             'clustering_avg': None
#         }
#     elif weight_matrix.ndim == 1:  # 1D array
#         nnz = np.count_nonzero(weight_matrix)
#         density = nnz / weight_matrix.size if weight_matrix.size > 0 else 0
#         return {
#             'density': density,
#             'nnz_min': None,
#             'nnz_max': None,
#             'nnz_avg': None,
#             'nnz_sd': None,
#             'bw_min': None,
#             'bw_max': None,
#             'bw_avg': None,
#             'bw_sd': None,
#             'scatter_avg': None,
#             'scatter_sd': None,
#             'clustering_avg': None
#         }

#     N = weight_matrix.shape[0]
#     nnz = np.count_nonzero(weight_matrix)
    
#     # Calculate density
#     if weight_matrix.ndim == 2:
#         density = nnz / (N ** 2) if N > 0 else 0
#     else:
#         density = nnz / weight_matrix.size

#     if weight_matrix.ndim == 2:
#         nnz_per_row = np.count_nonzero(weight_matrix, axis=1)
#         bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in weight_matrix]
#     elif weight_matrix.ndim == 4:
#         reshaped_matrix = weight_matrix.reshape(weight_matrix.shape[0], -1)
#         nnz_per_row = np.count_nonzero(reshaped_matrix, axis=1)
#         bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in reshaped_matrix]
#     else:
#         return {
#             'density': density,
#             'nnz_min': None,
#             'nnz_max': None,
#             'nnz_avg': None,
#             'nnz_sd': None,
#             'bw_min': None,
#             'bw_max': None,
#             'bw_avg': None,
#             'bw_sd': None,
#             'scatter_avg': None,
#             'scatter_sd': None,
#             'clustering_avg': None
#         }

#     nnz_min = np.min(nnz_per_row) if nnz_per_row.size > 0 else 0
#     nnz_max = np.max(nnz_per_row) if nnz_per_row.size > 0 else 0
#     nnz_avg = np.mean(nnz_per_row) if nnz_per_row.size > 0 else 0
#     nnz_sd = np.std(nnz_per_row) if nnz_per_row.size > 0 else 0

#     bw_min = np.min(bw) if bw else 0
#     bw_max = np.max(bw) if bw else 0
#     bw_avg = np.mean(bw) if bw else 0
#     bw_sd = np.std(bw) if bw else 0

#     scatter = nnz_per_row / (np.array(bw) + 1e-9)
#     scatter_avg = np.mean(scatter) if scatter.size > 0 else 0
#     scatter_sd = np.std(scatter) if scatter.size > 0 else 0

#     clustering = []
#     for row in reshaped_matrix if weight_matrix.ndim == 4 else weight_matrix:
#         groups = 0
#         prev_nonzero = -2
#         for idx in np.nonzero(row)[0]:
#             if idx != prev_nonzero + 1:
#                 groups += 1
#             prev_nonzero = idx
#         clustering.append(groups / (np.count_nonzero(row) + 1e-9))
#     clustering_avg = np.mean(clustering) if clustering else 0

#     return {
#         'density': density,
#         'nnz_min': nnz_min,
#         'nnz_max': nnz_max,
#         'nnz_avg': nnz_avg,
#         'nnz_sd': nnz_sd,
#         'bw_min': bw_min,
#         'bw_max': bw_max,
#         'bw_avg': bw_avg,
#         'bw_sd': bw_sd,
#         'scatter_avg': scatter_avg,
#         'scatter_sd': scatter_sd,
#         'clustering_avg': clustering_avg
#     }

# # Function to extract metrics from models
# def extract_metrics_from_models(model_stubs, sample_size=None):
#     metrics_list = []

#     for stub, accuracy in model_stubs:
#         print(f"Processing model: {stub}")
#         model = Model(stub)
#         onnx_model_file = model.onnx_model.path
#         onnx_model = onnx.load(onnx_model_file)

#         for tensor in onnx_model.graph.initializer:
#             weights = numpy_helper.to_array(tensor)
#             features = calculate_features(weights)

#             if any(value is None for value in features.values()):
#                 continue  # Skip this layer if any feature is None

#             features['layer'] = tensor.name
#             features['accuracy'] = accuracy
#             features['model'] = stub
#             metrics_list.append(features)
    
#     df_metrics = pd.DataFrame(metrics_list)
    
#     if sample_size and len(df_metrics) > sample_size:
#         df_metrics = df_metrics.sample(n=sample_size, random_state=42)  # Sample data to reduce size
    
#     return df_metrics

# # Function to plot the distribution of metrics
# def plot_metrics_distribution(df_metrics):
#     metrics = ['density', 'nnz_min', 'nnz_max', 'nnz_avg', 'nnz_sd', 'bw_min', 'bw_max', 'bw_avg', 'bw_sd', 'scatter_avg', 'scatter_sd', 'clustering_avg']
    
#     for metric in metrics:
#         plt.figure(figsize=(10, 6))
#         sns.histplot(df_metrics[metric].dropna(), kde=True, bins=50)
#         plt.title(f'Distribution of {metric}')
#         plt.xlabel(metric)
#         plt.ylabel('Frequency')
#         plt.show()

# # Function to plot correlation of metrics with accuracy
# def plot_correlation_with_accuracy(df_metrics):
#     metrics = ['density', 'nnz_min', 'nnz_max', 'nnz_avg', 'nnz_sd', 'bw_min', 'bw_max', 'bw_avg', 'bw_sd', 'scatter_avg', 'scatter_sd', 'clustering_avg']
    
#     for metric in metrics:
#         plt.figure(figsize=(10, 6))
#         sns.regplot(x=df_metrics[metric], y=df_metrics['accuracy'], ci=None, scatter_kws={'s': 50})
#         plt.title(f'Correlation of {metric} with Accuracy')
#         plt.xlabel(metric)
#         plt.ylabel('Accuracy')
#         plt.show()

# # Example model stubs with their corresponding accuracies
# model_stubs = [
#     ("zoo:resnet_v1-50-imagenet-pruned95_quantized", 75.8),
#     ("zoo:resnet_v1-18-imagenet-pruned85.4block_quantized", 69),
#     ("zoo:resnet_v1-34-imagenet-pruned", 73.3),
#     ("zoo:resnet_v1-50-imagenette-pruned",99.9),
#     ("zoo:resnet_v1-101-imagenet-pruned", 76.6),
#     ("zoo:resnet_v1-152-imagenet-pruned", 77.5),
#     ("zoo:resnet_v1-50_2xwidth-imagenet-base", 78.1),
#     ("zoo:resnet_v1-101_2xwidth-imagenet-base", 78.8)
#     # Add more models and their accuracies here
# ]

# # Extract metrics from the models
# df_metrics = extract_metrics_from_models(model_stubs, sample_size=1000)  # Add sample_size to limit data if necessary

# # Plot the distribution of metrics
# plot_metrics_distribution(df_metrics)

# # Plot the correlation of metrics with accuracy
# plot_correlation_with_accuracy(df_metrics)


# Second Run (ResNetV1 Models Variants)

In [75]:
import os
import onnx
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from onnx import numpy_helper
from sparsezoo import Model

# Function to calculate sparsity and features
def calculate_features(weight_matrix):
    if weight_matrix.size == 0:  # Check if the matrix size is zero
        return {
            'density': None,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    if weight_matrix.ndim == 0:  # Scalar
        return {
            'density': 1.0 if weight_matrix != 0 else 0.0,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }
    elif weight_matrix.ndim == 1:  # 1D array
        nnz = np.count_nonzero(weight_matrix)
        density = nnz / weight_matrix.size if weight_matrix.size > 0 else 0
        return {
            'density': density,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    N = weight_matrix.shape[0]
    nnz = np.count_nonzero(weight_matrix)
    
    # Calculate density
    if weight_matrix.ndim == 2:
        density = nnz / (N ** 2) if N > 0 else 0
    else:
        density = nnz / weight_matrix.size

    if weight_matrix.ndim == 2:
        nnz_per_row = np.count_nonzero(weight_matrix, axis=1)
        bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in weight_matrix]
    elif weight_matrix.ndim == 4:
        reshaped_matrix = weight_matrix.reshape(weight_matrix.shape[0], -1)
        nnz_per_row = np.count_nonzero(reshaped_matrix, axis=1)
        bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in reshaped_matrix]
    else:
        return {
            'density': density,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    nnz_min = np.min(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_max = np.max(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_avg = np.mean(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_sd = np.std(nnz_per_row) if nnz_per_row.size > 0 else 0

    bw_min = np.min(bw) if bw else 0
    bw_max = np.max(bw) if bw else 0
    bw_avg = np.mean(bw) if bw else 0
    bw_sd = np.std(bw) if bw else 0

    scatter = nnz_per_row / (np.array(bw) + 1e-9)
    scatter_avg = np.mean(scatter) if scatter.size > 0 else 0
    scatter_sd = np.std(scatter) if scatter.size > 0 else 0

    clustering = []
    for row in reshaped_matrix if weight_matrix.ndim == 4 else weight_matrix:
        groups = 0
        prev_nonzero = -2
        for idx in np.nonzero(row)[0]:
            if idx != prev_nonzero + 1:
                groups += 1
            prev_nonzero = idx
        clustering.append(groups / (np.count_nonzero(row) + 1e-9))
    clustering_avg = np.mean(clustering) if clustering else 0

    return {
        'density': density,
        'nnz_min': nnz_min,
        'nnz_max': nnz_max,
        'nnz_avg': nnz_avg,
        'nnz_sd': nnz_sd,
        'bw_min': bw_min,
        'bw_max': bw_max,
        'bw_avg': bw_avg,
        'bw_sd': bw_sd,
        'scatter_avg': scatter_avg,
        'scatter_sd': scatter_sd,
        'clustering_avg': clustering_avg
    }


def extract_metrics_from_models(model_stubs, sample_size=None):
    metrics_list = []

    for stub, accuracy in model_stubs:
        print(f"Processing model: {stub}")
        model = Model(stub)
        onnx_model_file = model.onnx_model.path
        onnx_model = onnx.load(onnx_model_file)

        for tensor in onnx_model.graph.initializer:
            weights = numpy_helper.to_array(tensor)
            features = calculate_features(weights)

            if any(value is None for value in features.values()):
                continue  # Skip this layer if any feature is None

            features['layer'] = tensor.name
            features['accuracy'] = accuracy
            features['model'] = stub
            metrics_list.append(features)
    
    df_metrics = pd.DataFrame(metrics_list)
    
    if sample_size and len(df_metrics) > sample_size:
        df_metrics = df_metrics.sample(n=sample_size, random_state=42)  # Sample data to reduce size
    
    return df_metrics


# Function to plot the distribution of metrics and save them as images
def plot_metrics_distribution(df_metrics, output_dir="plots_resnet"):
    metrics = ['density', 'nnz_min', 'nnz_max', 'nnz_avg', 'nnz_sd', 'bw_min', 'bw_max', 'bw_avg', 'bw_sd', 'scatter_avg', 'scatter_sd', 'clustering_avg']
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for metric in metrics:
        # Check if the column exists and has valid (non-None) data
        if metric in df_metrics.columns and df_metrics[metric].notna().any():
            plt.figure(figsize=(10, 6))
            sns.histplot(df_metrics[metric].dropna(), kde=True, bins=50)
            plt.title(f'Distribution of {metric}')
            plt.xlabel(metric)
            plt.ylabel('Frequency')
            plt.savefig(os.path.join(output_dir, f'{metric}_distribution.png'))
            plt.close()  # Close the plot to free memory

# Function to plot correlation of metrics with accuracy and save them as images
def plot_correlation_with_accuracy(df_metrics, output_dir="plots_resnet"):
    metrics = ['density', 'nnz_min', 'nnz_max', 'nnz_avg', 'nnz_sd', 'bw_min', 'bw_max', 'bw_avg', 'bw_sd', 'scatter_avg', 'scatter_sd', 'clustering_avg']
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for metric in metrics:
        # Check if the column exists and has valid (non-None) data
        if metric in df_metrics.columns and df_metrics[metric].notna().any():
            plt.figure(figsize=(10, 6))
            sns.regplot(x=df_metrics[metric], y=df_metrics['accuracy'], ci=None, scatter_kws={'s': 50})
            plt.title(f'Correlation of {metric} with Accuracy')
            plt.xlabel(metric)
            plt.ylabel('Accuracy')
            plt.savefig(os.path.join(output_dir, f'{metric}_correlation.png'))
            plt.close()  # Close the plot to free memory

# Example model stubs with their corresponding accuracies
model_stubs = [
    ("zoo:resnet_v1-18-imagenet-pruned85.4block_quantized", 69),
    ("zoo:resnet_v1-34-imagenet-pruned", 73.3),
    ("zoo:resnet_v1-50-imagenette-pruned",99.9),
    ("zoo:resnet_v1-101-imagenet-pruned", 76.6),
    ("zoo:resnet_v1-152-imagenet-pruned", 77.5),
    ("zoo:resnet_v1-50_2xwidth-imagenet-base", 78.1),
    ("zoo:resnet_v1-101_2xwidth-imagenet-base", 78.8)
    # Add more models and their accuracies here
]

# Extract metrics from the models
df_metrics = extract_metrics_from_models(model_stubs, sample_size=1000)  # Add sample_size to limit data if necessary

# Plot the distribution of metrics and save them as images
plot_metrics_distribution(df_metrics, output_dir="plots_resnet")

# Plot the correlation of metrics with accuracy and save them as images
plot_correlation_with_accuracy(df_metrics, output_dir="plots_resnet")



Processing model: zoo:resnet_v1-18-imagenet-pruned85.4block_quantized
Processing model: zoo:resnet_v1-34-imagenet-pruned
Processing model: zoo:resnet_v1-50-imagenette-pruned
Processing model: zoo:resnet_v1-101-imagenet-pruned
Processing model: zoo:resnet_v1-152-imagenet-pruned
Processing model: zoo:resnet_v1-50_2xwidth-imagenet-base
Processing model: zoo:resnet_v1-101_2xwidth-imagenet-base


In [76]:
import os

# List all files in the 'plots' directory
output_dir = "plots_resnet"
files = os.listdir(output_dir)
files


['bw_avg_correlation.png',
 'bw_avg_distribution.png',
 'bw_max_correlation.png',
 'bw_max_distribution.png',
 'bw_min_correlation.png',
 'bw_min_distribution.png',
 'bw_sd_correlation.png',
 'bw_sd_distribution.png',
 'clustering_avg_correlation.png',
 'clustering_avg_distribution.png',
 'density_correlation.png',
 'density_distribution.png',
 'nnz_avg_correlation.png',
 'nnz_avg_distribution.png',
 'nnz_max_correlation.png',
 'nnz_max_distribution.png',
 'nnz_min_correlation.png',
 'nnz_min_distribution.png',
 'nnz_sd_correlation.png',
 'nnz_sd_distribution.png',
 'scatter_avg_correlation.png',
 'scatter_avg_distribution.png',
 'scatter_sd_correlation.png',
 'scatter_sd_distribution.png']

In [77]:
import shutil

# Create a zip file containing all images
shutil.make_archive('plots_resnet_archive', 'zip', 'plots_resnet')


'C:\\Users\\ayush\\Desktop\\Msc\\plots_resnet_archive.zip'

In [78]:
from IPython.display import FileLink

# Create a download link for the zip file
FileLink('plots_resnet_archive.zip')


C:\Users\ayush\Desktop\Msc\plots_resnet_archive.zip

# Third Run (YOLOV5, YOLOV8, ResNetV1, VGG)

In [85]:
import os
import onnx
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from onnx import numpy_helper
from sparsezoo import Model

# Function to calculate sparsity and features
def calculate_features(weight_matrix):
    if weight_matrix.size == 0:  # Check if the matrix size is zero
        return {
            'density': None,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    if weight_matrix.ndim == 0:  # Scalar
        return {
            'density': 1.0 if weight_matrix != 0 else 0.0,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }
    elif weight_matrix.ndim == 1:  # 1D array
        nnz = np.count_nonzero(weight_matrix)
        density = nnz / weight_matrix.size if weight_matrix.size > 0 else 0
        return {
            'density': density,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    N = weight_matrix.shape[0]
    nnz = np.count_nonzero(weight_matrix)
    
    # Calculate density
    if weight_matrix.ndim == 2:
        density = nnz / (N ** 2) if N > 0 else 0
    else:
        density = nnz / weight_matrix.size

    if weight_matrix.ndim == 2:
        nnz_per_row = np.count_nonzero(weight_matrix, axis=1)
        bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in weight_matrix]
    elif weight_matrix.ndim == 4:
        reshaped_matrix = weight_matrix.reshape(weight_matrix.shape[0], -1)
        nnz_per_row = np.count_nonzero(reshaped_matrix, axis=1)
        bw = [np.max(np.nonzero(row)[0]) - np.min(np.nonzero(row)[0]) + 1 if np.count_nonzero(row) > 0 else 0 for row in reshaped_matrix]
    else:
        return {
            'density': density,
            'nnz_min': None,
            'nnz_max': None,
            'nnz_avg': None,
            'nnz_sd': None,
            'bw_min': None,
            'bw_max': None,
            'bw_avg': None,
            'bw_sd': None,
            'scatter_avg': None,
            'scatter_sd': None,
            'clustering_avg': None
        }

    nnz_min = np.min(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_max = np.max(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_avg = np.mean(nnz_per_row) if nnz_per_row.size > 0 else 0
    nnz_sd = np.std(nnz_per_row) if nnz_per_row.size > 0 else 0

    bw_min = np.min(bw) if bw else 0
    bw_max = np.max(bw) if bw else 0
    bw_avg = np.mean(bw) if bw else 0
    bw_sd = np.std(bw) if bw else 0

    scatter = nnz_per_row / (np.array(bw) + 1e-9)
    scatter_avg = np.mean(scatter) if scatter.size > 0 else 0
    scatter_sd = np.std(scatter) if scatter.size > 0 else 0

    clustering = []
    for row in reshaped_matrix if weight_matrix.ndim == 4 else weight_matrix:
        groups = 0
        prev_nonzero = -2
        for idx in np.nonzero(row)[0]:
            if idx != prev_nonzero + 1:
                groups += 1
            prev_nonzero = idx
        clustering.append(groups / (np.count_nonzero(row) + 1e-9))
    clustering_avg = np.mean(clustering) if clustering else 0

    return {
        'density': density,
        'nnz_min': nnz_min,
        'nnz_max': nnz_max,
        'nnz_avg': nnz_avg,
        'nnz_sd': nnz_sd,
        'bw_min': bw_min,
        'bw_max': bw_max,
        'bw_avg': bw_avg,
        'bw_sd': bw_sd,
        'scatter_avg': scatter_avg,
        'scatter_sd': scatter_sd,
        'clustering_avg': clustering_avg
    }


def extract_metrics_from_models(model_stubs, sample_size=None):
    metrics_list = []

    for stub, accuracy in model_stubs:
        print(f"Processing model: {stub}")
        model = Model(stub)
        onnx_model_file = model.onnx_model.path
        onnx_model = onnx.load(onnx_model_file)

        for tensor in onnx_model.graph.initializer:
            weights = numpy_helper.to_array(tensor)
            features = calculate_features(weights)

            if any(value is None for value in features.values()):
                continue  # Skip this layer if any feature is None

            features['layer'] = tensor.name
            features['accuracy'] = accuracy
            features['model'] = stub
            metrics_list.append(features)
    
    df_metrics = pd.DataFrame(metrics_list)
    
    if sample_size and len(df_metrics) > sample_size:
        df_metrics = df_metrics.sample(n=sample_size, random_state=42)  # Sample data to reduce size
    
    return df_metrics


# Function to plot the distribution of metrics and save them as images
def plot_metrics_distribution(df_metrics, output_dir="plots_new"):
    metrics = ['density', 'nnz_min', 'nnz_max', 'nnz_avg', 'nnz_sd', 'bw_min', 'bw_max', 'bw_avg', 'bw_sd', 'scatter_avg', 'scatter_sd', 'clustering_avg']
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for metric in metrics:
        # Check if the column exists and has valid (non-None) data
        if metric in df_metrics.columns and df_metrics[metric].notna().any():
            plt.figure(figsize=(10, 6))
            sns.histplot(df_metrics[metric].dropna(), kde=True, bins=50)
            plt.title(f'Distribution of {metric}')
            plt.xlabel(metric)
            plt.ylabel('Frequency')
            plt.savefig(os.path.join(output_dir, f'{metric}_distribution.png'))
            plt.close()  # Close the plot to free memory

# Function to plot correlation of metrics with accuracy and save them as images
def plot_correlation_with_accuracy(df_metrics, output_dir="plots_new"):
    metrics = ['density', 'nnz_min', 'nnz_max', 'nnz_avg', 'nnz_sd', 'bw_min', 'bw_max', 'bw_avg', 'bw_sd', 'scatter_avg', 'scatter_sd', 'clustering_avg']
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for metric in metrics:
        # Check if the column exists and has valid (non-None) data
        if metric in df_metrics.columns and df_metrics[metric].notna().any():
            plt.figure(figsize=(10, 6))
            sns.regplot(x=df_metrics[metric], y=df_metrics['accuracy'], ci=None, scatter_kws={'s': 50})
            plt.title(f'Correlation of {metric} with Accuracy')
            plt.xlabel(metric)
            plt.ylabel('Accuracy')
            plt.savefig(os.path.join(output_dir, f'{metric}_correlation.png'))
            plt.close()  # Close the plot to free memory

# Example model stubs with their corresponding accuracies
model_stubs = [
    ("zoo:yolov5-s6-coco-pruned65_quantized", 61.5),
    ("zoo:vgg-16-imagenet-pruned", 70.8),
    ("zoo:resnet_v1-34-imagenet-pruned", 73.3),
    ("zoo:yolov8-m-coco-pruned80_quantized", 65.2),
    ("zoo:yolov5-m6-coco-pruned75_quantized", 67.3),
    ("zoo:vgg-19-imagenet-pruned", 71.7),
    ("zoo:yolov5-s6-voc_coco-pruned65_quantized", 86.8),
    ("zoo:vgg-11-imagenet-pruned", 68.3),
    ("zoo:resnet_v1-18-imagenet-pruned85.4block_quantized", 69),
    ("zoo:yolov8-n-coco-pruned49_quantized", 50.1)
    # Add more models and their accuracies here
]

# Extract metrics from the models
df_metrics = extract_metrics_from_models(model_stubs, sample_size=1000)  # Add sample_size to limit data if necessary

# Plot the distribution of metrics and save them as images
plot_metrics_distribution(df_metrics, output_dir="plots_new")

# Plot the correlation of metrics with accuracy and save them as images
plot_correlation_with_accuracy(df_metrics, output_dir="plots_new")



Processing model: zoo:yolov5-s6-coco-pruned65_quantized
Processing model: zoo:vgg-16-imagenet-pruned
Processing model: zoo:resnet_v1-34-imagenet-pruned
Processing model: zoo:yolov8-m-coco-pruned80_quantized
Processing model: zoo:yolov5-m6-coco-pruned75_quantized
Processing model: zoo:vgg-19-imagenet-pruned
Processing model: zoo:yolov5-s6-voc_coco-pruned65_quantized
Processing model: zoo:vgg-11-imagenet-pruned
Processing model: zoo:resnet_v1-18-imagenet-pruned85.4block_quantized
Processing model: zoo:yolov8-n-coco-pruned49_quantized


In [86]:
import os

# List all files in the 'plots' directory
output_dir = "plots_new"
files = os.listdir(output_dir)
files


['bw_avg_correlation.png',
 'bw_avg_distribution.png',
 'bw_max_correlation.png',
 'bw_max_distribution.png',
 'bw_min_correlation.png',
 'bw_min_distribution.png',
 'bw_sd_correlation.png',
 'bw_sd_distribution.png',
 'clustering_avg_correlation.png',
 'clustering_avg_distribution.png',
 'density_correlation.png',
 'density_distribution.png',
 'nnz_avg_correlation.png',
 'nnz_avg_distribution.png',
 'nnz_max_correlation.png',
 'nnz_max_distribution.png',
 'nnz_min_correlation.png',
 'nnz_min_distribution.png',
 'nnz_sd_correlation.png',
 'nnz_sd_distribution.png',
 'scatter_avg_correlation.png',
 'scatter_avg_distribution.png',
 'scatter_sd_correlation.png',
 'scatter_sd_distribution.png']

In [87]:
import shutil

# Create a zip file containing all images
shutil.make_archive('plots_new_archive', 'zip', 'plots_new')


'C:\\Users\\ayush\\Desktop\\Msc\\plots_new_archive.zip'

In [88]:
from IPython.display import FileLink

# Create a download link for the zip file
FileLink('plots_new_archive.zip')


C:\Users\ayush\Desktop\Msc\plots_new_archive.zip